# Transformer Task Pipelines 

<a target="_blank" href="https://colab.research.google.com/github/raghavbali/llm_workshop/blob/main/module_01/solutions/03_explore_transformers.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## BERT-ology
- BERT, or __[Bi-Directional Encoder Representations from Transformers](https://arxiv.org/abs/1810.04805)__, was presented by Devlin et al., a team at Google AI in 2018
- Multi-task Learning: BERT also helped push the transfer-learning envelope in the NLP domain by showcasing how a pre-trained model can be fine-tuned for various tasks to provide state-of-the-art performance
- BERT tweaked the usual Language Model objective to only predict next token based on past context by building context from both directions, i.e. the objective of predicting masked words along with next sentence prediction.


<img src="../../assets/02_bert_models_layout_notebook_3.jpeg">

> source [PLM Papers](https://github.com/thunlp/PLMpapers)

In [6]:
import torch
import transformers
from transformers import pipeline

In [7]:
# Let us define some configs/constants
DISTILBET_BASE_UNCASED_CHECKPOINT = "distilbert/distilbert-base-uncased"
DISTILBET_QA_CHECKPOINT = "distilbert/distilbert-base-uncased-distilled-squad"
DISTILBET_CLASSIFICATION_CHECKPOINT = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"

In [8]:
if torch.cuda.is_available():
    DEVICE = 'cuda'
    Tensor = torch.cuda.FloatTensor
    LongTensor = torch.cuda.LongTensor
    DEVICE_ID = 0
elif torch.backends.mps.is_available():
    DEVICE = 'mps'
    Tensor = torch.FloatTensor
    LongTensor = torch.LongTensor
    DEVICE_ID = 0
else:
    DEVICE = 'cpu'
    Tensor = torch.FloatTensor
    LongTensor = torch.LongTensor
    DEVICE_ID = -1
print(f"Backend Accelerator Device={DEVICE}")

Backend Accelerator Device=mps


### Predicting the Masked Token
This was a unique objective when BERT was originally introduced as compared to usual NLP tasks such as classification. The objective requires us to prepare a dataset where we mask a certain percentage of input tokens and train the model to learn to predict those tokens. This objective turns out to be very effective in helping the model learn the nuances of language. 

In this first task we will test the pre-trained model against this objective itself. The model outputs a bunch of things such as the predicted token, encoded index of the predicted token/word along with a score which indicates the model's confidence.

In [9]:
mlm_pipeline = pipeline(
    'fill-mask',
    model=DISTILBET_BASE_UNCASED_CHECKPOINT,
    device=DEVICE_ID
)
mlm_pipeline("Bangalore is the IT [MASK] of India.")

/Users/raghavbali/.pyenv/versions/3.11.9/envs/datahack/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[{'score': 0.6093319058418274,
  'token': 3007,
  'token_str': 'capital',
  'sequence': 'bangalore is the it capital of india.'},
 {'score': 0.11960869282484055,
  'token': 9594,
  'token_str': 'hub',
  'sequence': 'bangalore is the it hub of india.'},
 {'score': 0.07699015736579895,
  'token': 2110,
  'token_str': 'state',
  'sequence': 'bangalore is the it state of india.'},
 {'score': 0.018147675320506096,
  'token': 11909,
  'token_str': 'gateway',
  'sequence': 'bangalore is the it gateway of india.'},
 {'score': 0.014192461967468262,
  'token': 4075,
  'token_str': 'headquarters',
  'sequence': 'bangalore is the it headquarters of india.'}]

### Question Answering
This is an interesting NLP task and quite complex one as well. For this task, the model is provided input consisting of the context along with a question and it predicts the answer by selecting text from the context. The training setup for this task is a bit involved process, the following is an overview:
- The training input as triplet of context, question and answer
- This is transformed as combined input of the form ``[CLS]question[SEP]context[SEP]`` or ``[CLS]contex[SEP]question[SEP]`` with answer acting as the label
- The model is trained to predict the start and end indices of the the corresponding answer for each input.


For our current setting, we will leverage both _pretrained_ and _fine-tuned_ versions of **DistilBERT** via the _question-answering_ pipeline and understand the performance difference.

In [10]:
qa_ft_pipeline = pipeline(
    'question-answering',
    model=DISTILBET_QA_CHECKPOINT,
    device=DEVICE_ID
)
qa_pt_pipeline = pipeline(
    'question-answering',
    model=DISTILBET_BASE_UNCASED_CHECKPOINT, #TODO: Set the pretrained 
    device=DEVICE_ID
)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# we use a snippet about BERT like models from the module itself
context = """The key contribution from this set of models is the masked language modeling objective during the pre-training phase, where some tokens in the input are masked, and the model is trained to predict them (we will cover these in the upcoming section). Key works in this group of architectures are BERT, RoBERTa (or optimized BERT), DistilBERT (lighter and more efficient BERT), ELECTRA and ALBERT.
In this notebook we will work through the task of Question Answering where our language model will learn to answer questions based on the context provided."""
question = "What are the key works in this set of models?"

In [12]:
ft_qa_result= qa_ft_pipeline(
    question=question,
    context=context
)

pt_qa_result= qa_pt_pipeline(
    question=question,
    context=context
)

In [13]:
print("*"*55)
print(f"Context:{context}")
print("*"*55)
print(f"Question:{question}")
print("-"*55)
print(f"Response from Fine-Tuned Model:\n{ft_qa_result}")
print()
print(f"Response from Pretrained Model:\n{pt_qa_result}")

*******************************************************
Context:The key contribution from this set of models is the masked language modeling objective during the pre-training phase, where some tokens in the input are masked, and the model is trained to predict them (we will cover these in the upcoming section). Key works in this group of architectures are BERT, RoBERTa (or optimized BERT), DistilBERT (lighter and more efficient BERT), ELECTRA and ALBERT.
In this notebook we will work through the task of Question Answering where our language model will learn to answer questions based on the context provided.
*******************************************************
Question:What are the key works in this set of models?
-------------------------------------------------------
Response from Fine-Tuned Model:
{'score': 0.010789199732244015, 'start': 294, 'end': 326, 'answer': 'BERT, RoBERTa (or optimized BERT'}

Response from Pretrained Model:
{'score': 0.0001713668170850724, 'start': 349, 'e

# Generative Pretraining

## Behold, its GPT (Generative pre-training)

The first model in this series is called GPT, or Generative Pre-Training. It was released in [2018](https://openai.com/blog/language-unsupervised/), about the same time as the BERT model. The paper presents a task-agnostic architecture based on the ideas of transformers and unsupervised learning.

- GPT is essentially a language model based on the __transformer-decoder__ 
- Introduction of large training datasets: __BookCorpus__ dataset contains over 7,000 unique, unpublished books across different genres
- The GPT architecture makes use of 12 decoder blocks (as opposed to 6 in the original transformer) with 768-dimensional states and 12 self-attention heads each.


### GPT-2
- Radford et al. presented the GPT-2 model as part of their work titled [Language Models are Unsupervised Multi-task Learners in 2019](https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)
- The model achieves state-of-the-art performance in a few-shot setting
- Similar to GPT, the secret sauce for GPT-2 is its dataset. The authors prepared a massive 40 GB dataset by crawling 45 million outbound links from a social networking site called Reddit.
- The vocabulary was also expanded to cover 50,000 words and the context window was expanded to 1,024 tokens (as compared to 512 for GPT).


### GPT-3
- OpenAI published paper titled [Language Models are Few Shot Learners](https://arxiv.org/abs/2005.14165) in May 2020. 
- This paper introduces the mammoth __175 billion-parameter GPT-3 model__.
- Apart from more layers and parameters, this model made use of sparse attention
- Dataset again played a key role, a 300 billion-token dataset based on existing datasets like Common Crawl (filtered for better content), WebText2 (a larger version of WebText used for GPT-2), Books1 and Books2, and the Wikipedia dataset was prepared for this model

## Language Modeling
By far the most widely used application from the NLP world is language modeling. We use it daily on our phone keyboards, email applications and a ton of other places.

In simple words, a language model takes certain text as input context to generate the next set of words as output. This is interesting because a language model tries to understand the input context, the language structure (though in a very naive way) to predict the next word(s). We use it in the form of text completion utilities on search engines, chat platforms, emails etc. all the time. Language models are a perfect real life application of NLP and showcase the power of RNNs.

Language models can be developed train in different ways. The most common and widely used method is the sliding window approach. The model takes a small window of text as input and tried to predict the next word as the output. The following figure illustrates the same visually.

<img src="../../assets/02_lm_training_notebook_3.png">

### PreTrained GPT2 for Text Generation

In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [15]:
# generative tasks are not available through MPS/Apple Silicon
DEVICE = 'cpu'
Tensor = torch.FloatTensor
LongTensor = torch.LongTensor
DEVICE_ID = -1
print(f"Backend Accelerator Device={DEVICE}")

Backend Accelerator Device=cpu


In [16]:
tokenizer = AutoTokenizer.from_pretrained("gpt2") #TODO: get pretrained GPT2 tokenizer

# add the EOS token as PAD token to avoid warnings
model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(DEVICE)

/Users/raghavbali/.pyenv/versions/3.11.9/envs/datahack/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
# encode context the generation is conditioned on
model_inputs = tokenizer('The king of England is', return_tensors='pt').to(DEVICE)

# generate 40 new tokens
greedy_output = model.generate(**model_inputs, max_new_tokens=40)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
The king of England is a man of great wealth and power, and he is a man of great wealth and power. He is a man of great wealth and power. He is a man of great wealth and power. He
